<a href="https://colab.research.google.com/github/faruknafizozbekir/northwind_psql/blob/master/Nisan_T%C3%BCm_Ma%C4%9Fazalar_Auto_ARIMASIZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Eğer bu kütüphaneler yüklü değilse çalıştırın:
!pip install pandas numpy openpyxl statsmodels tqdm

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Veri Setim.xlsx to Veri Setim.xlsx


In [9]:
# Excel dosyasının adını doğru yazın (örneğin: Veri Setim.xlsx)
file_name = "Veri Setim.xlsx"
# Excel dosyasının adını doğru yazın (örneğin: Veri Setim.xlsx)
file_name = "Veri Setim.xlsx"

# Sayfa isimlerini kontrol edin
import pandas as pd
excel_file = pd.ExcelFile(file_name)
print(excel_file)

In [10]:
df = pd.read_excel(excel_file, sheet_name="Sayfa1")
df.head()

,Mağaza Kod,202301,202302,202303,202304,202305,202306,202307,202308,202309,...,202406,202407,202408,202409,202410,202411,202412,202501,202502,202503
0,AL02,67187625.01,43436520.14,48067520.08,73385261.73,66369361.95,79903473.48,86993652.75,55191242.36,72017639.20,...,79692085.39,86298379.74,68466354.66,86164780.86,87594329.93,81312868.39,81403470.73,67350876.67,50831115.63,58573008.24
1,AL03,43440853.32,27591622.19,29786714.99,45746803.05,27387535.36,27763250.00,32815860.86,22882627.49,25637811.65,...,36508366.65,43023260.86,29642181.73,34730875.10,36260223.34,36083039.13,35146833.34,29354723.04,21775361.92,24717652.25
2,AL04,31660364.20,20175190.50,22712908.67,38536347.31,34971610.80,38713460.55,41335742.81,29372325.85,34672610.12,...,36883620.85,39532845.41,33217131.35,42252491.79,40882258.36,36510001.62,37739587.50,30124109.36,20978631.81,25155050.69
3,AL05,27740522.50,16761928.75,17073129.70,29028210.43,25561507.02,29703450.82,35447715.77,26248113.32,30436190.79,...,29496600.60,34928828.31,32479564.91,35445744.18,30822793.39,28752623.34,32898769.29,27519904.79,20796348.37,22624016.13
4,AL06,48597819.36,31583506.65,32616969.98,50159226.60,48603969.94,56120409.56,66724606.67,51547334.07,52343546.61,...,60379196.24,70488028.43,58409699.63,62693211.80,59007761.71,53774729.04,55201021.70,47269759.34,36700854.70,43678978.69


In [11]:
date_columns = df.columns[1:]

def improved_fill_missing(row):
    for i in range(len(date_columns)):
        if pd.isna(row[date_columns[i]]):
            prev_values = row[date_columns[max(0, i - 3):i]].dropna()
            if len(prev_values) > 0:
                row[date_columns[i]] = prev_values.mean()
            else:
                next_values = row[date_columns[i+1:i+4]].dropna()
                if len(next_values) > 0:
                    row[date_columns[i]] = next_values.mean()
    return row

df_filled = df.copy()
df_filled = df_filled.apply(improved_fill_missing, axis=1)

# Kalan boşlukları ileri ve geri doldur
df_filled[date_columns] = df_filled[date_columns].fillna(method='ffill', axis=1)
df_filled[date_columns] = df_filled[date_columns].fillna(method='bfill', axis=1)

# Tüm eksik veriler temizlendi mi kontrol
print("Kalan eksik değer sayısı:", df_filled.isnull().sum().sum())

Kalan eksik değer sayısı: 0


<ipython-input-11-f903ecca78d8>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled[date_columns] = df_filled[date_columns].fillna(method='ffill', axis=1)
<ipython-input-11-f903ecca78d8>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled[date_columns] = df_filled[date_columns].fillna(method='bfill', axis=1)


In [12]:
forecast_results = []

ordered_months = sorted(date_columns)  # Tarih kolonlarını sıralı al

for _, row in tqdm(df_filled.iterrows(), total=df_filled.shape[0]):
    store_code = row['Mağaza Kod']
    series = row[ordered_months]

    try:
        model = SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        forecast = results.forecast(steps=1)
        forecast_value = forecast.iloc[0]
    except:
        # Model başarısızsa son 3 ay ortalamasını kullan
        forecast_value = series[-3:].mean()

    forecast_results.append({
        'Mağaza Kod': store_code,
        'Tahmini_202504_Ciro': forecast_value
    })

forecast_df = pd.DataFrame(forecast_results)
forecast_df.head()

100%|██████████| 592/592 [00:02<00:00, 239.24it/s]


,Mağaza Kod,Tahmini_202504_Ciro
0,AL02,5.891833e+07
1,AL03,2.528258e+07
2,AL04,2.541926e+07
3,AL05,2.364676e+07
4,AL06,4.254986e+07


In [13]:
forecast_df.to_excel("202504_Tahminleri.xlsx", index=False)

# İndirmek için:
files.download("202504_Tahminleri.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Auto ARIMASIZ Tahmin